In [3]:
# Xin quyen Google drive
from google.colab import drive
drive.mount('/content/ggdrive')

Drive already mounted at /content/ggdrive; to attempt to forcibly remount, call drive.mount("/content/ggdrive", force_remount=True).


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, roc_auc_score
import warnings
warnings.filterwarnings('ignore')

# Đọc dữ liệu từ file CSV
def load_data():
    df = pd.read_csv("/content/ggdrive/MyDrive/Phân tích hiệu suất kinh doanh TIMA/test2.csv")
    return df

# Tiền xử lý dữ liệu
def preprocess_data(df):
    # Chọn các đặc trưng liên quan
    features = ['TS_CREDIT_SCORE_V2', 'Salary', 'Age', 'LoanDuration',
                'Gender', 'CityName', 'JobName', 'ReceiveYourIncomeSalary',
                'ProductCreditName', 'InterestPaymentType']
    target = 'HasLatePayment'

    # Kiểm tra và xác minh HasLatePayment với LongestOverdue
    df['HasLatePayment_verified'] = np.where(df['LongestOverdue'] > 0, 1, 0)
    if not (df['HasLatePayment'] == df['HasLatePayment_verified']).all():
        print("Warning: Discrepancy found between HasLatePayment and LongestOverdue")

    # Xử lý giá trị thiếu
    df['Age'].fillna(df['Age'].median(), inplace=True)
    df['Salary'].fillna(df['Salary'].median(), inplace=True)
    df['LoanDuration'].fillna(df['LoanDuration'].median(), inplace=True)
    df['TS_CREDIT_SCORE_V2'].fillna(df['TS_CREDIT_SCORE_V2'].median(), inplace=True)

    # Xử lý các cột phân loại
    categorical_cols = ['Gender', 'CityName', 'JobName', 'ReceiveYourIncomeSalary',
                        'ProductCreditName', 'InterestPaymentType']
    le = LabelEncoder()
    for col in categorical_cols:
        df[col] = df[col].fillna('Unknown')
        df[col] = le.fit_transform(df[col])

    # Chia dữ liệu thành X (đặc trưng) và y (mục tiêu)
    X = df[features]
    y = df[target]

    # Chuẩn hóa các đặc trưng số
    scaler = StandardScaler()
    numeric_cols = ['TS_CREDIT_SCORE_V2', 'Salary', 'Age', 'LoanDuration']
    X[numeric_cols] = scaler.fit_transform(X[numeric_cols])

    return X, y

# Huấn luyện và đánh giá mô hình
def train_and_evaluate_model(X, y):
    # Chia dữ liệu thành tập huấn luyện và kiểm tra
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Khởi tạo và huấn luyện mô hình Random Forest
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Dự đoán trên tập kiểm tra
    y_pred = model.predict(X_test)

    # Đánh giá mô hình
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    # Tính ROC-AUC
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    print(f"ROC-AUC Score: {roc_auc:.4f}")

    # Lấy độ quan trọng của đặc trưng
    feature_importance = pd.DataFrame({
        'Feature': X.columns,
        'Importance': model.feature_importances_
    }).sort_values(by='Importance', ascending=False)
    print("\nFeature Importance:")
    print(feature_importance)

    return model

# Hàm chính
def main():
    # Tải dữ liệu
    df = load_data()

    # Tiền xử lý dữ liệu
    X, y = preprocess_data(df)

    # Huấn luyện và đánh giá mô hình
    model = train_and_evaluate_model(X, y)

    return model

# Chạy chương trình
if __name__ == "__main__":
    model = main()

Classification Report:
              precision    recall  f1-score   support

           0       0.84      1.00      0.91       397
           1       1.00      0.03      0.05        80

    accuracy                           0.84       477
   macro avg       0.92      0.51      0.48       477
weighted avg       0.86      0.84      0.77       477

ROC-AUC Score: 0.6479

Feature Importance:
                   Feature  Importance
0       TS_CREDIT_SCORE_V2    0.226612
2                      Age    0.172096
1                   Salary    0.159354
3             LoanDuration    0.139801
6                  JobName    0.102553
8        ProductCreditName    0.069899
9      InterestPaymentType    0.037029
4                   Gender    0.033502
7  ReceiveYourIncomeSalary    0.030313
5                 CityName    0.028841
